In [13]:
import sys, os
# Add the 'scripts' directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from data_processing import process_data
# Step 1: Unzip the file
zip_file_path = '../Data'  # Replace with your .zip file path
extracted_folder_path = zip_file_path  # Replace with your desired extract path

data = process_data(zip_file_path+'/raw_analyst_ratings.csv.zip', 'raw_analyst_ratings.csv')